In [1]:
import sys,os
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

#this works apparently only for savefig stuff
mpl.rcParams['figure.figsize']=(15,10)    #(6.0,4.0)
mpl.rcParams['font.size']=10                #10 
mpl.rcParams['savefig.dpi']=400             #72 
mpl.rcParams['figure.subplot.bottom']=.1    #.125


plt.rc('font', family='serif')
plt.rc('text', usetex=False)

#inline Shit
#%matplotlib inline
%config InlineBackend.figure_format='svg'
%config InlineBackend.rc = {'figure.facecolor': 'white', 'figure.subplot.bottom': 0.125, 'figure.edgecolor': 'white', 'savefig.dpi': 400, 'figure.figsize': (12.0, 8.0), 'font.size': 10}

#GUi shit
%matplotlib tk

mpl.get_configdir()

%load_ext autoreload
%autoreload 2

In [182]:
import openpiv.tools
import openpiv.process
import openpiv.scaling
from skimage import img_as_int
    
deltaT=1e-3; # Framerate of the sequence

# frame_a  = img_as_int(openpiv.tools.imread('../../data/experiment1/experiment1bright_0320.jpg')).astype(np.int32) # male picture from float64 to int16 (try to use int32 FIX!)
# frame_b  = img_as_int(openpiv.tools.imread('../../data/experiment1/experiment1bright_0321.jpg')).astype(np.int32)
# frame_a  = img_as_int(openpiv.tools.imread('experiment1_bright_0428.jpg')).astype(np.int32) # male picture from float64 to int16 (try to use int32 FIX!)
# frame_b  = img_as_int(openpiv.tools.imread('experiment1_bright_0429.jpg')).astype(np.int32)
frame_a  = img_as_int(openpiv.tools.imread('../../data/experiment1/exp1-0781.jpg')).astype(np.int32) # male picture from float64 to int16 (try to use int32 FIX!)
frame_b  = img_as_int(openpiv.tools.imread('../../data/experiment1/exp1-0785.jpg')).astype(np.int32)


In [183]:
plt.matshow( frame_a, cmap=cm.Greys_r );
plt.matshow( frame_b, cmap=cm.Greys_r );
plt.show()

In [119]:
window_size= 16;
spacing = 8;
overlap = window_size-spacing;


u, v, sig2noise = openpiv.process.extended_search_area_piv(frame_a,
                                                           frame_b, 
                                                           window_size=window_size, 
                                                           overlap=overlap, 
                                                           dt=deltaT, 
                                                           search_area_size=16, 
                                                           sig2noise_method='peak2peak')
print(sig2noise)
x, y = openpiv.process.get_coordinates( image_size=frame_a.shape, window_size=window_size, overlap=overlap )

[[ 2.96330386  1.57142442  1.59879225 ...,  1.96249615  2.02275199  0.        ]
 [ 2.42426133  2.23100401  1.57644244 ...,  2.38874075  2.56524625  0.        ]
 [ 3.39791066  1.71500149  1.28206169 ...,  2.39014759  2.25292531  0.        ]
 ..., 
 [ 2.71744621  2.93759698  1.56484464 ...,  1.42762692  1.56492671  0.        ]
 [ 1.69851569  1.67956137  2.62563669 ...,  1.25335287  1.71118344  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [120]:
u2 = u.copy();
v2 = v.copy();
u2, v2, mask = openpiv.validation.sig2noise_val( u2, v2, sig2noise, threshold = 3);

In [121]:
u3 = u2.copy();
v3 = v2.copy();
u3, v3 = openpiv.filters.replace_outliers( u3, v3, method='localmean', max_iter=10, kernel_size=2 )

scaleFactor = 967.5931;
x_meter, y_meter, u3_meter, v3_meter = openpiv.scaling.uniform(x, y, u3, v3, scaling_factor = scaleFactor )

print(np.max(v3_meter))


1.08023989218


In [122]:
plt.close("all")

fig = plt.figure(figsize=(15,10));


ax0 = fig.add_subplot(131)
ax0.set_autoscale_on(False)
ax0.set_title("PIV Extended Search Area" )

quiveropts = {'scale_units' : 'inches', 'scale' : 4}

ax0.matshow( frame_a, cmap=cm.Greys_r );
ax0.quiver( x, y, u, v , color='r')

ax1 = fig.add_subplot(132, sharex=ax0, sharey=ax0)
ax1.set_autoscale_on(False)
ax1.set_title("Outlier Rejected" )

ax1.matshow( frame_a, cmap=cm.Greys_r );
ax1.quiver( x, y, u2, v2 , color='r')

ax3 = fig.add_subplot(133, sharex=ax0, sharey=ax0)
ax3.set_autoscale_on(False)
ax3.set_title("Replaced Outlier" )

ax3.matshow( frame_a, cmap=cm.Greys_r );
ax3.quiver( x, y, u3_meter, v3_meter , color='r', **quiveropts)


ax0.set_xlim([0,frame_a.shape[1]])
ax0.set_ylim([0,frame_a.shape[0]])
ax1.set_xlim([0,frame_a.shape[1]])
ax1.set_ylim([0,frame_a.shape[0]])
fig.tight_layout()
plt.show()

fig = plt.figure(figsize=(15,10));
ax0 = fig.add_subplot(111)
p = ax0.imshow( sig2noise, cmap=cm.coolwarm);
cbar = fig.colorbar(p)

In [186]:
window_size= 32;
spacing = 8;
overlap = window_size-spacing;
deltaT = 1.0
u, v, sig2noise = openpiv.pyprocess.piv(frame_a, 
                                        frame_b, 
                                        window_size=window_size, 
                                        overlap=overlap, 
                                        dt=deltaT, 
                                        corr_method='fft',
                                        sig2noise_method='peak2peak')

x, y = openpiv.process.get_coordinates( image_size=frame_a.shape, window_size=window_size, overlap=overlap )

u2 = u.copy();
v2 = v.copy();
u2, v2, mask = openpiv.validation.sig2noise_val( u2, v2, sig2noise, threshold = 1.05);

In [187]:

fig = plt.figure(figsize=(15,10));

opts = {"units":'dots', "angles":'xy', "scale_units":'xy', "scale":1.0};

ax0 = fig.add_subplot(121)
ax0.set_autoscale_on(False)

ax0.matshow( frame_a, cmap=cm.Greys_r );
ax0.quiver( x, y, u, v , color='r',**opts)
ax0.set_title("PIV Extended Search Area" )

ax1 = fig.add_subplot(122, sharex=ax0, sharey=ax0)
ax1.set_autoscale_on(False)
ax1.set_title("Outlier Rejected" )
ax1.matshow( frame_b, cmap=cm.Greys_r );
ax1.quiver( x, y, u2, v2 , color='r', **opts)


ax0.set_xlim([0,frame_a.shape[1]])
ax0.set_ylim([0,frame_a.shape[0]])
ax1.set_xlim([0,frame_a.shape[1]])
ax1.set_ylim([0,frame_a.shape[0]])
fig.tight_layout()
plt.show()